## Zadanie domowe - Algorytm Canny'ego

Celem zadania domowego jest wykonanie pełnej implementacji algorytmu Canny'ego.

W ramach ćwiczenia w trakcie laboratorium wyznaczono obrazy $g_{NH}$ i $g_{NL}$.
Dla przypomnienia:
Można powiedzieć, że na obrazie $g_{NH}$ są "pewne" krawędzie.
Natomiast na $g_{NL}$ "potencjalne".
Często krawędzie "pewne" nie są ciągłe.
Wykorzystuje się więc krawędzie "potencjalne", aby uzupełnić nieciągłości.
Procedura wygląda następująco:
1. Stwórz stos zawierający wszystkie piksele zaznaczone na obrazie $g_{NH}$.
W tym celu wykorzystaj listę współrzędnych `[row, col]`.
Do pobrania elementu z początku służy metoda `list.pop()`.
Do dodania elementu na koniec listy służy metoda `list.append(new)`.
2. Stwórz obraz, który będzie zawierał informację czy dany piksel został już odwiedzony.
3. Stwórz obraz, który zawierać będzie wynikowe krawędzie.
Jej rozmiar jest równy rozmiarowi obrazu.
4. Wykonaj pętlę, która będzie pobierać elementy z listy, dopóki ta nie będzie pusta.
W tym celu najlepiej sprawdzi się pętla `while`.
    - W każdej iteracji pobierz element ze stosu.
    - Sprawdź, czy dany element został już odwiedzony.
    - Jeśli nie został, to:
        - Oznacz go jako odwiedzony,
        - Oznacz piksel jako krawędź w wyniku,
        - Sprawdź otoczenie piksela w obrazie $g_{NL}$,
        - Dodaj do stosu współrzędne otoczenia, które zawierają krawędź.
        Można to wykonać np. pętlą po stworzonym otoczeniu.
7. Wyświetl obraz oryginalny, obraz $g_{NH}$ oraz obraz wynikowy.
8. Porównaj wynik algorytmu z wynikiem OpenCV.

Pomocnicze obrazy $g_{NH}$ i $g_{NL}$ zostały wprowadzone dla uproszczenia opisu.
Algorytm można zaimplementować w bardziej "zwarty" sposób.

Na podstawie powyższego opisu zaimplementuj pełny algorytm Canny'ego.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os

if not os.path.exists("dom.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/09_Canny/dom.png --no-check-certificate

dom = cv2.imread("dom.png", cv2.IMREAD_GRAYSCALE)
plt.gray()
plt.imshow(dom)

def d(x):
    if x == 0 or x==4:
        return 0, 1, 0, -1
    elif x == -1 or x==3:
        return 1, 1, -1, -1
    elif x == -2 or x==2:
        return 0, 1, 0, -1
    elif x == -3 or x==1:
        return 1, -1, -1, 1

def qnt(x):
    if x < 0:
        return qnt(x + 180)
    if x <= 22.5:
        return 1
    elif x <= 67.5:
        return 2
    elif x <= 112.5:
        return 3
    elif x <= 157.5:
        return 4
    else:
        return 1

def uncanny(src: np.ndarray, th, tl):
    blured = cv2.GaussianBlur(src, (3, 3), 3).astype('float')
    hy = 0.25 * np.array([[1, 2, 1],
                        [0, 0, 0],
                        [-1, -2, -1]]).astype('float')
    hx = 0.25 * np.array([[1, 0, -1],
                        [2, 0, -2],
                        [1, 0, -1]]).astype('float')

    gx = cv2.filter2D(src=blured, ddepth=-3, kernel=hx, dst=None)
    gy = cv2.filter2D(src=blured, ddepth=-3, kernel=hy, dst=None)

    M = np.sqrt(np.add(np.square(gx), np.square(gy)))
    alfa = np.arctan2(gy, gx)

    ans = M.copy()
    for y in range(1, src.shape[0]-1):
        for x in range(1, src.shape[1]-1):
            dn = d(qnt(alfa[y, x]))
            if M[y + dn[0], x + dn[1]] > M[y, x] or M[y + dn[2], x + dn[3]] > M[y, x]:
                ans[y, x] = 0


    gh = np.where(ans >= th, 0, 1)
    gl = np.where(np.logical_and(ans >= tl, ans < th), 0, 1)
    
    return gl, gh


In [ ]:

def fullcanny(src: np.ndarray, th, tl):
    gl, gh = uncanny(src, th, tl)
    gl = (-1*gl + 1).astype(bool)
    gh = (-1*gh + 1).astype(bool)
    lst_h = []
    for row in range(1, gh.shape[0]-1):
        for col in range(1, gh.shape[1]-1):
            if gh[row, col]:
                lst_h.append((row, col))
    visited = np.zeros(gh.shape).astype(bool)
    ans = np.zeros(gl.shape).astype(bool)
    while len(lst_h):
        cords = lst_h.pop()
        if not visited[cords[0], cords[1]]:
            visited[cords[0], cords[1]] = True
            ans[cords[0], cords[1]] = True
            for row in range(cords[0]-2, cords[0]+ 1):
                for col in range(cords[1]-2, cords[1]+ 1):
                    if 0<row<gh.shape[0] and 0<col<gh.shape[1]:
                        if gl[row, col] and not visited[row, col]:
                            lst_h.append((row, col))
    return ans.astype('uint8')

x = fullcanny(dom, 30, 12)
plt.figure()
plt.imshow(x)
x = cv2.Canny(dom, 180, 250, None, 3, 1)
plt.figure()
plt.imshow(x)

#Obie metody dają bardzo podobne wyniki, wbudowane canny np może brać pod uwagę inne okno